#### Часть 2. Проверка гипотез методами математической статистики

В этой части группового проекта вам предстоит самим выбрать предметную область, а также самим получить датасет или датасеты, с которыми вы будете работать.

С точки зрения источников данных у вас есть две опции: вы можете найти ваш датасет — или 2-3 датасета в случае необходимости — на сайте [UCI Machine Learning Repository](https://archive.ics.uci.edu/) (брать датасеты необходимо именно с этого сайта); либо же вторая опция — можно собрать датасет/датасеты вручную с помощью средств Parsing'a и API. Допускается использование и комбинации этих двух опций. В любом случае, *каждый* используемый вами датасет должен (до этапа EDA) содержать **не менее 2000 строк** и **не менее 8 признаков**.




После получения датасетов вам следует:
- провести EDA с осмысленными визуализациями и качественным поиском первичных закономерностей;
- на основе проделенного EDA сформулировать **минимум три чёткие, содержательные гипотезы**;
- подобрать и реализовать **минимум три различных статистических теста** для проверки сформулированных раннее гипотез, причём каждый тест должен быть обязательно обоснован (различные статистические тесты в нашем случае означает, что статистические критерии для этих тестов должны быть разными);
- корректно и полно проинтерпретировать полученные результаты и сформулировать итоговые выводы.

**Важно:** вы должны очень ответственно подойти к выбору датасета, так как гипотезы должны быть *не случайными*, а логически вытекающими из осуществленного EDA и, кроме того, имеющими реальный интерес для изучения и проверки с точки зрения бизнес-составляющей. В случае если по одному датасету у вас не получается сформировать 3 достаточно осмысленных гипотезы — разрешается использовать суммарно вплоть до 3 датасетов. В таком случае, все эти датасеты должны иметь какую-то связь между собой.

In [ ]:
from selenium import webdriver
import time
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import MoveTargetOutOfBoundsException, StaleElementReferenceException
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import random

Настройка Selenium

In [ ]:

options = webdriver.ChromeOptions()
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/131.0.0.0 Safari/537.36"
)


options.add_argument("--window-size=900,700")


options.add_argument("--disable-gpu")
options.add_argument("--headless=new")

# не нужны расширения, уведомления, всплывашки
options.add_argument("--disable-extensions")
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")

# чтобы оптимизировать работу
options.add_argument("--disable-background-timer-throttling")
options.add_argument("--disable-renderer-backgrounding")
options.add_argument("--disable-backgrounding-occluded-windows")


# чтобы не грузить картинки
prefs = {
    "profile.managed_default_content_settings.images": 2
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

Колонки в будущем датафрейме

In [ ]:
columns = [
    'name',
    'link',
    'monWT',
    'tueWT',
    'wedWT',
    'thuWT',
    'friWT',
    'satWT',
    'sunWT',
    'avgBill',
    'rating',
    'rateAmount',
    'address',
    'distMetro',
    'metro',
    'photoAmount',
    'menuPositions',
    "reviewsAmount",
    "cuisine",
    "tablesAmount",
    "phoneListed",
    "reservationButton",
    "categories"

]

df = pd.DataFrame(columns=columns)

Вспомогательные функции, чтобы получать признаки

In [ ]:
def get_reviews_amount(soup): #количество отзывов
    el = soup.select_one('div[role="tab"][aria-label^="Отзывы"], ''div.tabs-select-view__title_name_reviews')
    if not el:
        return None

    label = el.get("aria-label") or el.get_text(strip=True)
    return _clean_int(label)

def _clean_int(text): # впомогательная функция для красивого вывода
    if not text:
        return None
    text = text.replace("\u00a0", " ")
    m = re.search(r"\d+", text)
    return int(m.group()) if m else None


def _format_avg_bill_raw(text): # форматирование среднего чека
    """
    '1 000 ₽'       -> '1000'
    '1 000–1 500 ₽' -> '1000-1500'
    """
    if not text:
        return None
    text = text.replace("\u00a0", " ")
    nums = re.findall(r"\d+", text)
    if not nums:
        return None
    if len(nums) == 1:
        return nums[0]
    return f"{nums[0]}-{nums[1]}"



def get_name(soup): #название ресторана
    h1 = soup.select_one('h1[itemprop="name"]')
    if h1 and h1.contents:
        return h1.contents[0].strip()
    return None


def get_rating(soup): # оценка ресторана
    el = soup.select_one("span.business-rating-badge-view__rating-text")
    if not el:
        return None
    txt = el.get_text(strip=True).replace(",", ".")
    try:
        return float(txt)
    except ValueError:
        return None


def get_rate_amount(soup): # количество оценок
    el = soup.select_one("div.business-header-rating-view__text._clickable")
    if not el:
        return 0

    aria = el.get("aria-label") or ""
    val = _clean_int(aria)
    if val is not None:
        return val

    txt = el.get_text(strip=True)
    return _clean_int(txt)


def get_address(soup): # адрес
    el = soup.select_one("a.business-contacts-view__address-link")
    return el.get_text(strip=True) if el else None


def get_metro_name(soup): # ближайшее метро
    block = soup.select_one("div.masstransit-stops-view__stop._type_metro")
    if not block:
        return None
    name_el = block.select_one("div.masstransit-stops-view__stop-name")
    return name_el.get_text(strip=True) if name_el else None


def get_dist_metro(soup): # расстояние до ближайшего метро
    block = soup.select_one("div.masstransit-stops-view__stop._type_metro")
    if not block:
        return None

    dist_el = block.select_one("div.masstransit-stops-view__stop-distance-text")
    if not dist_el:
        return None

    txt = dist_el.get_text(strip=True)
    txt = txt.replace("\u00a0", " ")

    return txt


def has_menu(soup): # вспомогательная функция для проверки наличия меню на сайте
    if soup.select_one("div.tabs-select-view__title._name_menu"):
        return True
   
    return False


def get_avg_bill(soup): # получение среднего чека
    for block in soup.select("div.business-features-view__valued"):
        title_el = block.select_one("span.business-features-view__valued-title")
        if not title_el:
            continue
        title_txt = title_el.get_text(strip=True)
        if "Средний счёт" not in title_txt:
            continue

        value_el = block.select_one("span.business-features-view__valued-value")
        if not value_el:
            return None
        value_txt = value_el.get_text(strip=True)
        return _format_avg_bill_raw(value_txt)
    return None


def get_photo_amount(soup): # количество фоток
    el = soup.select_one('div[role="tab"][aria-label^="Фото"]')
    if not el:
        el = soup.select_one("div.tabs-select-view__title_name_media")
    if not el:
        for cand in soup.select('div[role="tab"]'):
            label = cand.get("aria-label", "")
            if "Фото" in label:
                el = cand
                break
    if not el:
        return 0

    label = el.get("aria-label") or el.get_text(strip=True)
    return _clean_int(label)



def get_html_worktime(driver): # получить график работы
    try:
        time.sleep(random.uniform(1, 2))
        elem = driver.find_element(By.CSS_SELECTOR, "div.business-working-status-flip-view__control")

        actions = ActionChains(driver)
        actions.move_to_element(elem).click().perform()
        time.sleep(random.uniform(2, 3))

        new_html = driver.page_source

        actions = ActionChains(driver)
        actions.send_keys(Keys.ESCAPE).perform()

        return new_html
    except NoSuchElementException:
        return None
    except WebDriverException:
        return None


def parse_working_time(soup):  # время работы по дням недели
    day_map = {
        "Понедельник": "monWT",
        "Вторник": "tueWT",
        "Среда": "wedWT",
        "Четверг": "thuWT",
        "Пятница": "friWT",
        "Суббота": "satWT",
        "Воскресенье": "sunWT",
    }

    res = {
    "monWT": None,
    "tueWT": None,
    "wedWT": None,
    "thuWT": None,
    "friWT": None,
    "satWT": None,
    "sunWT": None,
    }
    
    if not soup:
        return res

    for row in soup.select("div.business-working-intervals-view__item"):
        day_el = row.select_one("div.business-working-intervals-view__day")
        intervals_el = row.select_one("div.business-working-intervals-view__intervals")
        if not day_el:
            continue

        day_name = day_el.get_text(strip=True)
        col_name = day_map.get(day_name)
        if not col_name:
            continue

        res[col_name] = intervals_el.get_text(" ", strip=True) if intervals_el else None

    return res



def get_menu_items_count(driver): # сколько блюд есть в карточке ресторана
    menu_tab = driver.find_element(By.CSS_SELECTOR, 'div[role="tab"][aria-label^="Меню"], ''div.tabs-select-view__title._name_menu')
    actions = ActionChains(driver)
    time.sleep(random.uniform(1, 2))
    actions.move_to_element(menu_tab).click().perform()
    time.sleep(random.uniform(1, 2))

    dishes = driver.find_elements(By.CSS_SELECTOR, "div.related-item-photo-view__main")
    return len(dishes)



def open_features_and_get_soup(driver): # получить суп особенностей
    try:
        features_tab = driver.find_element(By.CSS_SELECTOR, 'div[role="tab"][aria-label^="Особенности"], ''div.tabs-select-view__title_name_features')
        actions = ActionChains(driver)
        time.sleep(1)
        actions.move_to_element(features_tab).click().perform()
        time.sleep(random.uniform(1, 2))  # даём вкладке прогрузиться

        html = driver.page_source
        ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        soup = BeautifulSoup(html, "html.parser")
        return soup
    except:
        return None


def get_feature_value(soup, label_part): # вспомогательная функция внутри особеннсотей
    if soup is None:
        return None

    # каждый ряд вида: [Заголовок] [Значение]
    for block in soup.select("div.business-features-view__valued"):
        title_el = block.select_one(".business-features-view__valued-title")
        value_el = block.select_one(".business-features-view__valued-value")
        if not title_el or not value_el:
            continue

        title_text = title_el.get_text(" ", strip=True).lower()
        if label_part.lower() in title_text:
            return value_el.get_text(" ", strip=True)

    return None


def get_cuisine(soup): # какие кухни представлены в ресторане
    return get_feature_value(soup, "кухня")


def get_tables_seats(soup): # сколько есть столов
    return get_feature_value(soup, "количество столов")

def has_show_phone_button(soup): # указан ли в карточке ресторана номер телефона
    if soup is None:
        return False

    btn = soup.find(
        "span",
        string=lambda s: s and "Показать телефон" in s
    )
    return btn is not None

def has_reservation_button(soup): # есть ли возможность забронировать на сайте
    if soup is None:
        return False

    page_text = soup.get_text(" ", strip=True).lower()
    return "забронировать столик" in page_text


def get_categories(soup): # какие еще категории у ресторана
    if soup is None:
        return None

    
    block = soup.select_one("div.orgpage-categories-info-view")
    if block:
        names = []
        for span in block.select("span.button__text"):
            txt = span.get_text(strip=True)
            if txt:
                names.append(txt)
        if names:
            return ", ".join(names)

    return None

Основная функция

In [ ]:
def get_info(url):
    global df

    driver.get(url)
    time.sleep(random.uniform(3, 4))

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    
    wt_html = get_html_worktime(driver)
    wt_soup = BeautifulSoup(wt_html, "html.parser") if wt_html else None
    wt = parse_working_time(wt_soup)

    osobennosti_soup = open_features_and_get_soup(driver)
    
    if has_menu(soup):
        menu_positions = get_menu_items_count(driver)
    else:
        menu_positions = 0

    row = {
        "name": get_name(soup),
        "link": url,
        "monWT": wt.get("monWT"),
        "tueWT": wt.get("tueWT"),
        "wedWT": wt.get("wedWT"),
        "thuWT": wt.get("thuWT"),
        "friWT": wt.get("friWT"),
        "satWT": wt.get("satWT"),
        "sunWT": wt.get("sunWT"),
        "avgBill": get_avg_bill(soup),
        "rating": get_rating(soup),
        "rateAmount": get_rate_amount(soup),
        "address": get_address(soup),
        "distMetro": get_dist_metro(soup),
        "metro": get_metro_name(soup),
        "photoAmount": get_photo_amount(soup),
        "menuPositions": menu_positions,
        "reviewsAmount": get_reviews_amount(soup),
        "cuisine": get_cuisine(osobennosti_soup),
        "tablesAmount": get_tables_seats(osobennosti_soup),
        "phoneListed": has_show_phone_button(soup),
        "reservationButton": has_reservation_button(soup),
        "categories": get_categories(osobennosti_soup)
    }

    df.loc[len(df)] = row
    return row

In [ ]:
toRead = pd.read_csv("final_df_links_part1.csv")

In [223]:
toRead = toRead["url"]
toRead

0       https://yandex.ru/maps/org/abu_gosh/238185678950/
1       https://yandex.ru/maps/org/al33_pitstseriya/17...
2       https://yandex.ru/maps/org/ambassadori_trampli...
3        https://yandex.ru/maps/org/amphora/128434812078/
4       https://yandex.ru/maps/org/anderson/104269030501/
                              ...                        
2350    https://yandex.com/maps/org/china_today/113703...
2351    https://yandex.com/maps/org/coffeemania/102595...
2352    https://yandex.com/maps/org/coffeemania/104660...
2353    https://yandex.com/maps/org/coffeemania/106931...
2354    https://yandex.com/maps/org/coffeemania/107517...
Name: url, Length: 2355, dtype: object

In [ ]:
for i in toRead:
    raw_data = get_info(i)
    print(raw_data)

df.to_csv("processedRestaurants", index=False)